# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
%%capture
!pip install 'dlt[duckdb]'
!pip install pandas numpy

# 1. Use a generator

In [4]:
sum = 0
for value in square_root_generator(5):
    sum += value
print('sum of the outputs of the generator for limit = 5:', sum)

last_value = 0
for i, value in enumerate(square_root_generator(13)):
    if i + 1 == 13:
        last_value = value

print('13th number yielded:', last_value)


sum of the outputs of the generator for limit = 5: 8.382332347441762
13th number yielded: 3.605551275463989


# 2. Append a generator to a table with existing data

In [5]:
import dlt
import duckdb

pipeline = dlt.pipeline(pipeline_name='append-people', destination='duckdb', dataset_name='people_data')

info = pipeline.run(people_1(), table_name="people", write_disposition="replace")
print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print(f'{pipeline.pipeline_name} Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people_gen1_df = conn.sql("SELECT * FROM people").df()
display(people_gen1_df)
sum_people_gen1_age = people_gen1_df['age'].sum()
print('sum of ages of all people gen 1:', sum_people_gen1_age)

info = pipeline.run(people_2(), table_name="people", write_disposition="append")
print(info)

appended_people_df = conn.sql("SELECT * FROM people").df()
display(appended_people_df)
sum_all_appended_people_age = appended_people_df['age'].sum()
print('sum of ages of all appended people:', sum_all_appended_people_age)


Pipeline append-people load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/mhack/Projects/data-engineering/DataTalksClub/de-zoomcamp/de-zoomcamp-2024-dlt-hw/append-people.duckdb location to store data
Load package 1707279121.096697 is LOADED and contains no failed jobs
append-people Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707279121.096697,gSB+up8GBWgcOA
1,2,Person_2,27,City_A,1707279121.096697,9TUY6t8YaoHcag
2,3,Person_3,28,City_A,1707279121.096697,Ttvb/N5A5NFkKw
3,4,Person_4,29,City_A,1707279121.096697,V/bfI3/OYZ6Z9A
4,5,Person_5,30,City_A,1707279121.096697,chM4OTTpFfL7JA


sum of ages of all people gen 1: 140
Pipeline append-people load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/mhack/Projects/data-engineering/DataTalksClub/de-zoomcamp/de-zoomcamp-2024-dlt-hw/append-people.duckdb location to store data
Load package 1707279121.744777 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707279121.096697,gSB+up8GBWgcOA,None
1,2,Person_2,27,City_A,1707279121.096697,9TUY6t8YaoHcag,None
2,3,Person_3,28,City_A,1707279121.096697,Ttvb/N5A5NFkKw,None
3,4,Person_4,29,City_A,1707279121.096697,V/bfI3/OYZ6Z9A,None
4,5,Person_5,30,City_A,1707279121.096697,chM4OTTpFfL7JA,None
5,3,Person_3,33,City_B,1707279121.744777,5/6rZrqsFmDkog,Job_3
6,4,Person_4,34,City_B,1707279121.744777,0Wk1KUhZZ3rdZQ,Job_4
7,5,Person_5,35,City_B,1707279121.744777,O0RT9MlXtW98kw,Job_5
8,6,Person_6,36,City_B,1707279121.744777,tATLSBPWiyinOA,Job_6
9,7,Person_7,37,City_B,1707279121.744777,7yCY2/OBBZkWbw,Job_7


sum of ages of all appended people: 353


# 3. Merge a generator

In [6]:
pipeline = dlt.pipeline(pipeline_name='merge-people', destination='duckdb', dataset_name='people_data')

info = pipeline.run(people_1(), table_name="people", write_disposition="replace", primary_key='id')
print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print(f'{pipeline.pipeline_name} Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n people table below:")

people_gen1_df = conn.sql("SELECT * FROM people").df()
display(people_gen1_df)

info = pipeline.run(people_2(), table_name="people", write_disposition="merge", primary_key='id')
print(info)

merged_people_df = conn.sql("SELECT * FROM people").df()
display(merged_people_df)
sum_all_merged_people_age = merged_people_df['age'].sum()
print('sum of ages of all merged people:', sum_all_merged_people_age)


Pipeline merge-people load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/mhack/Projects/data-engineering/DataTalksClub/de-zoomcamp/de-zoomcamp-2024-dlt-hw/merge-people.duckdb location to store data
Load package 1707279122.351243 is LOADED and contains no failed jobs
merge-people Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707279122.351243,mVH8TPDyuwyTSA
1,2,Person_2,27,City_A,1707279122.351243,yAG+JLsWJTAHvA
2,3,Person_3,28,City_A,1707279122.351243,1UlwMwGuHe3oBw
3,4,Person_4,29,City_A,1707279122.351243,tT7QdR8xemyIOQ
4,5,Person_5,30,City_A,1707279122.351243,xXb9l47uZsmq8A


Pipeline merge-people load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/mhack/Projects/data-engineering/DataTalksClub/de-zoomcamp/de-zoomcamp-2024-dlt-hw/merge-people.duckdb location to store data
Load package 1707279122.9740531 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707279122.351243,mVH8TPDyuwyTSA,None
1,2,Person_2,27,City_A,1707279122.351243,yAG+JLsWJTAHvA,None
2,7,Person_7,37,City_B,1707279122.9740531,ZDq3Jt2NUzejxw,Job_7
3,8,Person_8,38,City_B,1707279122.9740531,eRbGuy6RI7xJjw,Job_8
4,4,Person_4,34,City_B,1707279122.9740531,1No8SwYgBnZTpg,Job_4
5,3,Person_3,33,City_B,1707279122.9740531,SV6DtJDVFQ+Svw,Job_3
6,6,Person_6,36,City_B,1707279122.9740531,YZMoYqKyPkirNg,Job_6
7,5,Person_5,35,City_B,1707279122.9740531,7NGXTi9ugMAn9Q,Job_5


sum of ages of all merged people: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX